In [1]:
import os 
os.getcwd()

'C:\\Users\\Alireza\\Desktop\\App'

In [2]:
import pandas as pd
import numpy as np
from sklearn import cluster, covariance, manifold

from datetime import datetime, timedelta 
import math


from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, DBSCAN
from scipy.cluster.hierarchy import fcluster, dendrogram, linkage, cophenet, ward
from scipy.spatial.distance import pdist
from sklearn.metrics import adjusted_mutual_info_score
from sklearn import cluster, covariance, manifold
from sklearn import metrics

import matplotlib.pyplot as plt
import matplotlib.dates as mdates


import warnings
warnings.filterwarnings('ignore')

In [3]:
dataset = pd.read_csv('dataset.csv')

In [4]:
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset.set_index('Date', inplace=True)

In [5]:
dataset.shape

(1258, 502)

In [6]:
dataset.tail(3)

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-27,139.820007,13.99,193.149994,154.880005,136.550003,109.790001,73.540001,353.820007,596.080017,199.350006,...,90.629997,61.540001,101.660004,35.630001,114.379997,130.839996,121.129997,275.790009,44.509998,196.899994
2023-12-28,139.770004,13.98,193.580002,154.750000,137.000000,110.400002,74.059998,351.589996,595.520020,200.240005,...,91.760002,61.889999,100.190002,35.590000,114.320000,130.520004,121.629997,275.350006,44.660000,197.160004
2023-12-29,139.029999,13.74,192.529999,154.970001,136.139999,110.070000,74.269997,350.910004,596.599976,198.559998,...,91.110001,61.910000,99.980003,35.590000,114.360001,130.660004,121.699997,273.329987,43.869999,197.369995


In [7]:
def dataCleaner(dataset):
    missing_fraction = dataset.isnull().mean().sort_values(ascending=False)
    
    drop_list = sorted(list(missing_fraction[missing_fraction > 0.1].index))
    
    dataset.drop(labels = drop_list, axis = 1, inplace = True)
    
    dataset.fillna(method = 'bfill', axis = 'index', inplace = True)
    
    print("Any null values?",dataset.isnull().values.any())
    return dataset

In [8]:
dataset = dataCleaner(dataset)

Any null values? False


In [9]:
def returnAndVolatility(dataset, startDate = '2019-01-01', endDate = '2023-12-29'):
    dataset = dataset.loc[startDate:endDate,]
    nDays = len(dataset)
    returns = dataset.pct_change().mean() * nDays
    returns = pd.DataFrame(returns)
    returns.columns = ['Returns']
    returns['Volatility'] = dataset.pct_change().std() * np.sqrt(nDays)
    data = returns
    return data

In [10]:
data = returnAndVolatility(dataset, '2019-01-01', '2023-12-29')
X = data

In [11]:
def selectAssets(X, n_clusters = '', method = '', checkRet = 0):
    
    if method == 'km':
        k_means = cluster.KMeans(n_clusters = n_clusters, random_state = 42)
        k_means.fit(X)
        labels = k_means.predict(X)
    
    elif method == 'hc':
        hc = AgglomerativeClustering(n_clusters = n_clusters, affinity = 'euclidean', linkage = 'ward')
        labels = hc.fit_predict(X)
    
    else:
        ap = AffinityPropagation()
        ap.fit(X)
        labels = ap.predict(X)
    
    # Get the indices of the stocks in each cluster
    clusters = {}
    for i, label in enumerate(labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(i)

    # Build the portfolio by selecting the stock with the highest Sharpe ratio in each cluster
    portfolio = []
    for label, indices in clusters.items():
        max_sharpe_ratio = -1
        best_stock = None
        for i in indices:
            ret = X.iloc[i, 0]
            vol = X.iloc[i, 1]
            
            if checkRet == True:
                if ret > 0:
                    sharpe_ratio = ret / vol
                    if sharpe_ratio > max_sharpe_ratio:
                        max_sharpe_ratio = sharpe_ratio
                        best_stock = i
            else:
                sharpe_ratio = ret / vol
                if sharpe_ratio > max_sharpe_ratio:
                    max_sharpe_ratio = sharpe_ratio
                    best_stock = i
        portfolio.append(best_stock)

    return portfolio

In [12]:
selectAssets(X, 5,'km')

[106, 265, 377, 475, 335]

In [29]:
X.iloc[335]

Returns       3.345993
Volatility    1.156703
Name: NVDA, dtype: float64

In [12]:
from pypfopt.efficient_frontier import EfficientFrontier 
from pypfopt import risk_models 
from pypfopt import expected_returns

from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt import objective_functions
from pypfopt import plotting

In [13]:
m1 = ['2019-01-01', '2022-12-29', '2023-01-30', '2023-02-28', '2023-03-30', '2023-04-28', '2023-05-30', '2023-06-30',
      '2023-07-28', '2023-08-30', '2023-09-29', '2023-10-30', '2023-11-30', '2023-12-29']

In [14]:
m2 = ['2019-01-01', '2022-12-29', '2023-02-28', '2023-04-28', '2023-06-30', '2023-08-30', '2023-10-30', '2023-12-29']

In [15]:
m3 = ['2019-01-01', '2022-12-29', '2023-03-30', '2023-06-30', '2023-09-29', '2023-12-29']

In [16]:
m4 = ['2019-01-01', '2022-12-29', '2023-04-28', '2023-08-30', '2023-12-29']

In [17]:
m6 = ['2019-01-01', '2022-12-29', '2023-06-30', '2023-12-29']

In [18]:
def Portfolio_Optimization(dataset, portfolio = ''):
    if portfolio == '':
        df = dataset
    else:
        portfolio = [x for x in portfolio if x is not None]
        df = dataset.iloc[:,portfolio]
    #Calculate the expected returns and the annualised sample covariance matrix of asset returns
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)
    ef = EfficientFrontier(mu, S)
    #optimize for max sharpe ratio
    
    ef.add_objective(objective_functions.L2_reg, gamma = 1.1)
    #ef.add_constraint(lambda w: w @ mu >= 0.9)
    #ef.add_constraint(lambda w: w.T @ S @ w <= 0.4 ** 2)
    
    
    weights = ef.max_sharpe()

    cleaned_weights = ef.clean_weights()
    cleaned_weights = {asset: round(weight, 2) for asset, weight in weights.items() if weight > 0.0}
    #print(cleaned_weights) 
    #print('Number of Assets:',  len(cleaned_weights))
    #ef.portfolio_performance(verbose = True)
    return weights

In [19]:
def plot(portfolio = ''):
    if portfolio == '':
        df = dataset
    else:
        portfolio = [x for x in portfolio if x is not None]
        df = dataset.iloc[:,portfolio]
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)
    ef = EfficientFrontier(mu, S)


    fig, ax = plt.subplots()
    ef_max_sharpe = ef.deepcopy()
    plotting.plot_efficient_frontier(ef, ax=ax, show_assets=True)

    # Find the tangency portfolio
    ef_max_sharpe.max_sharpe()
    ret_tangent, std_tangent, _ = ef_max_sharpe.portfolio_performance()
    ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Max Sharpe")

    # Generate random portfolios
    n_samples = 10000
    w = np.random.dirichlet(np.ones(ef.n_assets), n_samples)
    rets = w.dot(ef.expected_returns)
    stds = np.sqrt(np.diag(w @ ef.cov_matrix @ w.T))
    sharpes = rets / stds
    ax.scatter(stds, rets, marker=".", c=sharpes, cmap="viridis_r")

    # Output
    ax.set_title("Efficient Frontier with random portfolios")
    ax.legend()
    plt.tight_layout()
    #plt.savefig("ef_scatter.png", dpi=200)
    plt.show()

In [20]:
def SharpeRatio(dataset, portfolio = ''):
    if portfolio == '':
        df = dataset
    else:
        portfolio = [x for x in portfolio if x is not None]
        df = dataset.iloc[:, portfolio]

    # Calculate the expected returns and the annualized sample covariance matrix of asset returns
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)

    ef = EfficientFrontier(mu, S)  # Set the solver here

    weights = ef.max_sharpe()

    cleaned_weights = ef.clean_weights()
    cleaned_weights = {asset: round(weight, 2) for asset, weight in weights.items() if weight > 0.01}
    #print(cleaned_weights) 
    #print('Number of Assets:',  len(cleaned_weights))
    # to exit Sharp Ratio r is that tuple
    r = []
    r = ef.portfolio_performance()
    return r[2] 

In [21]:
def bestSharpeRatioIndex(dataset, endDate, startDate = '2019-01-01', method = ''):
    
        dataset = dataset.loc[startDate:endDate, ]
        if method == 'all':
            portfolio = ''
            sharpe = SharpeRatio(dataset)
        if method == '' or method == 'km' or method == 'hc':
            data = returnAndVolatility(dataset, startDate, endDate)
            if method == 'km' or method == 'hc':
                bestSharpeRatio = 0
                numberCluster = 0

                for n_cluster in range(10, 40):
                    portfolio = selectAssets(data, n_cluster, method)
                    bestSharpeRatioStep = SharpeRatio(dataset, portfolio)

                    if bestSharpeRatioStep >= bestSharpeRatio:
                        bestSharpeRatio = bestSharpeRatioStep
                        numberCluster = n_cluster

                #print ('Best numberCluster:', numberCluster)
                portfolio = selectAssets(data, numberCluster, method)
                sharpe = SharpeRatio(dataset, portfolio)
            if method == '':
                portfolio = selectAssets(data)
                sharpe = SharpeRatio(dataset, portfolio)
        return round(sharpe, 3)

In [22]:
def RebalanceSharpe(dataset, period, method):
    sharpeRatio = []
    for i in range(len(period) - 1):
        sharpe = bestSharpeRatioIndex(dataset, method = method, endDate = period[i + 1])
        sharpeRatio.append(sharpe)
    return sharpeRatio

In [23]:
def Discrete_Allocation(dataset, weights, portfolio = '', portfolio_val = 5000):
    if portfolio == '':
        df = dataset
    else:
        portfolio = [x for x in portfolio if x is not None]
        df = dataset.iloc[:, portfolio]
    
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(weights, latest_prices, total_portfolio_value = portfolio_val)
    allocation, leftover = da.lp_portfolio()
    print('Discrete allocation:', allocation)
    print('Funds Remaining:', round(leftover, 2)) 

In [24]:
def Discrete_Allocation2(dataset, weights, portfolio = '', portfolio_val = 5000):
    if portfolio == '':
        df = dataset
    else:
        portfolio = [x for x in portfolio if x is not None]
        df = dataset.iloc[:, portfolio]
    
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(weights, latest_prices, total_portfolio_value = portfolio_val)
    allocation, leftover = da.lp_portfolio()
    
    return allocation, round(leftover, 2)

In [25]:
def portfolioIncome(dataset, dic, date, leftover):   
    assets = [key for key, _ in dic.items()]    
    assetsNumber = list(dic.values())
    
    value = np.dot(dataset[assets].loc[date].values, assetsNumber)
    
    income = math.trunc(value + leftover)
    return income

In [26]:
def bestPortfolioIndex(dataset, endDate, portfolio_val = 5000, startDate = '2019-01-01', method = ''):
    
        
        dataset = dataset.loc[startDate:endDate, ]
        if method == 'all':
            portfolio = ''
            weights = Portfolio_Optimization(dataset)
            Discrete_Allocation(dataset, weights, portfolio, portfolio_val)
        else:
            data = returnAndVolatility(dataset, startDate, endDate)
            bestSharpeRatio = 0
            numberCluster = 0

            for n_cluster in range(10, 40):
                portfolio = selectAssets(data, n_cluster, method)
                bestSharpeRatioStep = SharpeRatio(dataset, portfolio)
                
                if bestSharpeRatioStep >= bestSharpeRatio:
                    bestSharpeRatio = bestSharpeRatioStep
                    numberCluster = n_cluster
                    
            print ('Best numberCluster:', numberCluster)
            portfolio = selectAssets(data, numberCluster, method)
            weights = Portfolio_Optimization(dataset, portfolio)
            Discrete_Allocation(dataset, weights, portfolio, portfolio_val)

In [27]:
def best(dataset, endDate, portfolio_val = 5000, startDate = '2019-01-01', method = ''):
    
        
        dataset = dataset.loc[startDate:endDate, ]
        if method == 'all':
            portfolio = ''
            weights = Portfolio_Optimization(dataset)
            allocation, leftover = Discrete_Allocation2(dataset, weights, portfolio, portfolio_val)
        if method == '' or method == 'km' or method == 'hc':
            data = returnAndVolatility(dataset, startDate, endDate)
            if method == 'km' or method == 'hc':
                bestSharpeRatio = 0
                numberCluster = 0

                for n_cluster in range(10, 40):
                    portfolio = selectAssets(data, n_cluster, method)
                    bestSharpeRatioStep = SharpeRatio(dataset, portfolio)

                    if bestSharpeRatioStep >= bestSharpeRatio:
                        bestSharpeRatio = bestSharpeRatioStep
                        numberCluster = n_cluster

                #print ('Best numberCluster:', numberCluster)
                portfolio = selectAssets(data, numberCluster, method)
                weights = Portfolio_Optimization(dataset, portfolio)
                allocation, leftover = Discrete_Allocation2(dataset, weights, portfolio, portfolio_val)
            if method == '':
                portfolio = selectAssets(data)
                weights = Portfolio_Optimization(dataset, portfolio)
                allocation, leftover = Discrete_Allocation2(dataset, weights, portfolio, portfolio_val)
        return allocation, leftover

In [28]:
def portfolioRebalanceIncome(dataset, Initial, period, method):
    portfolio_val = [Initial]
    for i in range(len(period) - 2):
        allocation, leftover = best(dataset, portfolio_val = portfolio_val[i], endDate = period[i + 1], method = method)
        pIncome = portfolioIncome(dataset, allocation, period[i + 2], leftover)
        portfolio_val.append(pIncome)
    return portfolio_val

In [29]:
resultsIncome = {
    'km': {
        'm1': [1000, 1007, 1020, 1011, 929, 1064, 1075, 1115, 1066, 1012, 844, 991, 1110],
        'm2': [1000, 1024, 917, 1030, 976, 823, 1066],
        'm3': [1000, 1029, 1089, 974, 1069],
        'm4': [1000, 957, 999, 998],
        'm6': [1000, 1112, 1103]
    },
    'hc': {
        'm1': [1000, 1007, 1014, 1012, 896, 1083, 1162, 1145, 1096, 1042, 883, 1026, 1130],
        'm2': [1000, 1033, 930, 1215, 1132, 898, 1092],
        'm3': [1000, 1102, 1272, 1120, 1168],
        'm4': [1000, 1062, 1335, 1384],
        'm6': [1000, 1267, 1243]
    },
    'af': {
        'm1': [1000, 972, 995, 982, 817, 958, 1011, 995, 928, 876, 765, 874, 943],
        'm2': [1000, 948, 832, 996, 919, 739, 909],
        'm3': [1000, 967, 815, 770, 870],
        'm4': [1000, 938, 919, 908],
        'm6': [1000, 1014, 1016]
    },
    'all': {
        'm1': [1000, 960, 964, 964, 889, 1042, 1063, 1100, 1071, 1005, 871, 987, 1104],
        'm2': [1000, 936, 863, 1054, 1067, 837, 1056],
        'm3': [1000, 949, 993, 908, 1019],
        'm4': [1000, 906, 918, 927],
        'm6': [1000, 953, 899]
    }
}

In [30]:
resultsSharpe = {
    'km': {
        'm1': [2.414, 2.104, 2.074, 2.108, 1.956, 2.0, 1.995, 1.961, 1.956, 1.851, 1.694, 1.774, 1.862],
        'm2': [2.414, 2.074, 1.956, 1.995, 1.956, 1.694, 1.862],
        'm3': [2.414, 2.108, 1.995, 1.851, 1.862],
        'm4': [2.414, 1.956, 1.956, 1.862],
        'm6': [2.414, 1.995, 1.862]
    },
    'hc': {
        'm1': [2.404, 2.119, 1.991, 1.972, 1.898, 1.953, 1.977, 1.958, 1.956, 1.834, 1.689, 1.763, 1.856],
        'm2': [2.404, 1.991, 1.898, 1.977, 1.956, 1.689, 1.856],
        'm3': [2.404, 1.972, 1.977, 1.834, 1.856],
        'm4': [2.404, 1.898, 1.956, 1.856],
        'm6': [2.404, 1.977, 1.856]
    },
    'ap': {
        'm1': [2.404, 2.119, 1.991, 1.972, 1.898, 1.953, 1.977, 1.958, 1.956, 1.834, 1.689, 1.763, 1.856],
        'm2': [2.404, 1.991, 1.898, 1.977, 1.956, 1.689, 1.856],
        'm3': [2.404, 1.972, 1.977, 1.834, 1.856],
        'm4': [2.404, 1.898, 1.956, 1.856],
        'm6': [2.404, 1.977, 1.856]
    },
    'all': {
        'm1': [2.42, 2.216, 2.142, 2.116, 2.018, 2.041, 2.049, 2.01, 1.996, 1.882, 1.744, 1.837, 1.903],
        'm2': [2.42, 2.142, 2.018, 2.049, 1.996, 1.744, 1.903],
        'm3': [2.42, 2.116, 2.049, 1.882, 1.903],
        'm4': [2.42, 2.018, 1.996, 1.903],
        'm6': [2.42, 2.049, 1.903]
    }
}

In [31]:
def ReturnVolatility(dataset, portfolio = ''):
    if portfolio == '':
        df = dataset
    else:
        portfolio = [x for x in portfolio if x is not None]
        df = dataset.iloc[:, portfolio]

    # Calculate the expected returns and the annualized sample covariance matrix of asset returns
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)

    ef = EfficientFrontier(mu, S)  # Set the solver here
    
    weights = ef.max_sharpe()

    cleaned_weights = ef.clean_weights()
    cleaned_weights = {asset: round(weight, 2) for asset, weight in weights.items() if weight > 0.01}
    #print(cleaned_weights) 
    #print('Number of Assets:',  len(cleaned_weights))
    # to exit Sharp Ratio r is that tuple
    r = []
    r = ef.portfolio_performance()
    Return, Volatility = r[0:2]
    return Return, Volatility

In [32]:
def ReturnVolatilityIndex(dataset, endDate, startDate = '2019-01-01', method = ''):
    
        dataset = dataset.loc[startDate:endDate, ]
        if method == 'all':
            portfolio = ''
            Return, Volatility = ReturnVolatility(dataset)
        if method == '' or method == 'km' or method == 'hc':
            data = returnAndVolatility(dataset, startDate, endDate)
            if method == 'km' or method == 'hc':
                bestSharpeRatio = 0
                numberCluster = 0

                for n_cluster in range(10, 40):
                    portfolio = selectAssets(data, n_cluster, method)
                    bestSharpeRatioStep = SharpeRatio(dataset, portfolio)

                    if bestSharpeRatioStep >= bestSharpeRatio:
                        bestSharpeRatio = bestSharpeRatioStep
                        numberCluster = n_cluster

                #print ('Best numberCluster:', numberCluster)
                portfolio = selectAssets(data, numberCluster, method)
                Return, Volatility = ReturnVolatility(dataset, portfolio)
            if method == '':
                portfolio = selectAssets(data)
                Return, Volatility = ReturnVolatility(dataset, portfolio)
        return Return, Volatility

In [33]:
def RebalanceReturnVolatility(dataset, period, method):
    d = len(period) - 2
    Return_list = []
    Volatility_list = []
    for i in range(len(period) - 1):
        current_return, current_volatility = ReturnVolatilityIndex(dataset, method=method, endDate=period[i + 1])
        
        Return_list.append(round(current_return / d, 3))
        Volatility_list.append(round(current_volatility / d, 3))
    return Return_list, Volatility_list

In [34]:
resultsReturn = {
    'km': {
        'm1': [0.064, 0.058, 0.054, 0.051, 0.047, 0.049, 0.047, 0.045, 0.047, 0.043, 0.04, 0.041, 0.042],
        'm2': [0.128, 0.108, 0.093, 0.093, 0.093, 0.079, 0.085],
        'm3': [0.192, 0.154, 0.14, 0.13, 0.127],
        'm4': [0.256, 0.186, 0.187, 0.169],
        'm6': [0.384, 0.279, 0.254]
    },
    'hc': {
        'm1': [0.064, 0.073, 0.07, 0.057, 0.046, 0.051, 0.05, 0.046, 0.047, 0.045, 0.04, 0.04, 0.043],
        'm2': [0.128, 0.139, 0.092, 0.101, 0.093, 0.08, 0.085],
        'm3': [0.193, 0.172, 0.151, 0.136, 0.128],
        'm4': [0.257, 0.185, 0.187, 0.171],
        'm6': [0.385, 0.302, 0.256]
    },
    'ap': {
        'm1': [0.064, 0.073, 0.07, 0.118, 0.047, 0.063, 0.052, 0.046, 0.049, 0.045, 0.04, 0.04, 0.043],
        'm2': [0.128, 0.139, 0.093, 0.104, 0.097, 0.081, 0.085],
        'm3': [0.193, 0.353, 0.156, 0.134, 0.128],
        'm4': [0.257, 0.187, 0.195, 0.171],
        'm6': [0.385, 0.313, 0.256]
    },
    'all': {
        'm1': [0.063, 0.056, 0.053, 0.05, 0.042, 0.046, 0.047, 0.047, 0.047, 0.044, 0.038, 0.04, 0.043],
        'm2': [0.125, 0.106, 0.085, 0.094, 0.094, 0.076, 0.086],
        'm3': [0.188, 0.15, 0.141, 0.131, 0.129],
        'm4': [0.25, 0.17, 0.187, 0.172],
        'm6': [0.375, 0.281, 0.258]
    }
}

In [35]:
resultsVolatility = {
    'km': {
        'm1': [0.026, 0.026, 0.025, 0.024, 0.023, 0.024, 0.023, 0.022, 0.023, 0.023, 0.022, 0.022, 0.022],
        'm2': [0.052, 0.05, 0.046, 0.045, 0.046, 0.045, 0.044],
        'm3': [0.077, 0.071, 0.068, 0.068, 0.066],
        'm4': [0.103, 0.092, 0.092, 0.087],
        'm6': [0.155, 0.135, 0.131]
    },
    'hc': {
        'm1': [0.026, 0.034, 0.034, 0.028, 0.023, 0.025, 0.025, 0.022, 0.023, 0.024, 0.023, 0.022, 0.022],
        'm2': [0.052, 0.068, 0.047, 0.049, 0.046, 0.045, 0.044],
        'm3': [0.078, 0.085, 0.074, 0.071, 0.066],
        'm4': [0.104, 0.094, 0.092, 0.088],
        'm6': [0.156, 0.148, 0.133]
    },
    'ap': {
        'm1': [0.026, 0.034, 0.034, 0.066, 0.023, 0.033, 0.026, 0.022, 0.024, 0.023, 0.023, 0.023, 0.022],
        'm2': [0.052, 0.068, 0.046, 0.051, 0.049, 0.046, 0.044],
        'm3': [0.078, 0.198, 0.077, 0.07, 0.067],
        'm4': [0.104, 0.092, 0.097, 0.089],
        'm6': [0.156, 0.153, 0.133]
    },
    'all': {
        'm1': [0.025, 0.025, 0.024, 0.023, 0.02, 0.022, 0.022, 0.022, 0.023, 0.022, 0.021, 0.021, 0.022],
        'm2': [0.05, 0.048, 0.04, 0.044, 0.045, 0.042, 0.043],
        'm3': [0.075, 0.069, 0.066, 0.067, 0.065],
        'm4': [0.101, 0.081, 0.091, 0.087],
        'm6': [0.151, 0.132, 0.13]
    }
}

In [36]:
import cvxpy as cp

def transaction_cost_objective(w, S, **kwargs):
    # Extract transaction_cost_pct from kwargs
    transaction_cost_pct = kwargs["transaction_cost_pct"]
    # Calculate the transaction cost term
    transaction_cost = transaction_cost_pct * cp.sum(cp.abs(w - 0.001))
    # Return the total objective function value
    return w @ S @ w + transaction_cost

In [37]:
def Portfolio_OptimizationT(dataset, portfolio = '', transaction_cost_pct=0.001):
    if portfolio == '':
        df = dataset
    else:
        portfolio = [x for x in portfolio if x is not None]
        df = dataset.iloc[:,portfolio]
    #Calculate the expected returns and the annualised sample covariance matrix of asset returns
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)
    ef = EfficientFrontier(mu, S)
    #optimize for max sharpe ratio
    
    ef.add_objective(objective_functions.L2_reg, gamma = 1.1)
    #ef.add_constraint(lambda w: w @ mu >= 0.9)
    #ef.add_constraint(lambda w: w.T @ S @ w <= 0.4 ** 2)
    
    ef.add_objective(transaction_cost_objective, S=S, transaction_cost_pct=transaction_cost_pct)
    
    weights = ef.max_sharpe()

    cleaned_weights = ef.clean_weights()
    cleaned_weights = {asset: round(weight, 2) for asset, weight in weights.items() if weight > 0.0}
    #print(cleaned_weights) 
    #print('Number of Assets:',  len(cleaned_weights))
    #ef.portfolio_performance(verbose = True)
    return weights

In [38]:
def Discrete_AllocationT(dataset, weights, portfolio = '', portfolio_val = 5000):
    if portfolio == '':
        df = dataset
    else:
        portfolio = [x for x in portfolio if x is not None]
        df = dataset.iloc[:, portfolio]
    
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(weights, latest_prices, total_portfolio_value = portfolio_val)
    allocation, leftover = da.lp_portfolio()
    print('Discrete allocation:', allocation)
    print('Funds Remaining:', round(leftover, 2)) 

In [39]:
def Discrete_Allocation2T(dataset, weights, portfolio = '', portfolio_val = 5000):
    if portfolio == '':
        df = dataset
    else:
        portfolio = [x for x in portfolio if x is not None]
        df = dataset.iloc[:, portfolio]
    
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(weights, latest_prices, total_portfolio_value = portfolio_val)
    allocation, leftover = da.lp_portfolio()
    
    return allocation, round(leftover, 2)

In [40]:
def SharpeRatioT(dataset, portfolio = '', transaction_cost_pct=0.01):
    if portfolio == '':
        df = dataset
    else:
        portfolio = [x for x in portfolio if x is not None]
        df = dataset.iloc[:, portfolio]

    # Calculate the expected returns and the annualized sample covariance matrix of asset returns
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)

    ef = EfficientFrontier(mu, S)  # Set the solver here
    
    ef.add_objective(transaction_cost_objective, S=S, transaction_cost_pct=transaction_cost_pct)
    
    weights = ef.max_sharpe()

    cleaned_weights = ef.clean_weights()
    cleaned_weights = {asset: round(weight, 2) for asset, weight in weights.items() if weight > 0.01}
    #print(cleaned_weights) 
    #print('Number of Assets:',  len(cleaned_weights))
    # to exit Sharp Ratio r is that tuple
    r = []
    r = ef.portfolio_performance()
    return r[2] 

In [41]:
def bestT(dataset, endDate, portfolio_val = 5000, startDate = '2019-01-01', method = ''):
    
        
        dataset = dataset.loc[startDate:endDate, ]
        if method == 'all':
            portfolio = ''
            weights = Portfolio_OptimizationT(dataset)
            allocation, leftover = Discrete_Allocation2T(dataset, weights, portfolio, portfolio_val)
        if method == '' or method == 'km' or method == 'hc':
            data = returnAndVolatility(dataset, startDate, endDate)
            if method == 'km' or method == 'hc':
                bestSharpeRatio = 0
                numberCluster = 0

                for n_cluster in range(10, 40):
                    portfolio = selectAssets(data, n_cluster, method)
                    bestSharpeRatioStep = SharpeRatio(dataset, portfolio)

                    if bestSharpeRatioStep >= bestSharpeRatio:
                        bestSharpeRatio = bestSharpeRatioStep
                        numberCluster = n_cluster

                #print ('Best numberCluster:', numberCluster)
                portfolio = selectAssets(data, numberCluster, method)
                weights = Portfolio_OptimizationT(dataset, portfolio)
                allocation, leftover = Discrete_Allocation2T(dataset, weights, portfolio, portfolio_val)
            if method == '':
                portfolio = selectAssets(data)
                weights = Portfolio_OptimizationT(dataset, portfolio)
                allocation, leftover = Discrete_Allocation2T(dataset, weights, portfolio, portfolio_val)
        return allocation, leftover

In [42]:
def portfolioRebalanceIncomeT(dataset, Initial, period, method):
    portfolio_val = [Initial]
    for i in range(len(period) - 2):
        allocation, leftover = bestT(dataset, portfolio_val = portfolio_val[i], endDate = period[i + 1], method = method)
        pIncome = portfolioIncome(dataset, allocation, period[i + 2], leftover)
        portfolio_val.append(pIncome)
    return portfolio_val

In [43]:
def bestSharpeRatioIndexT(dataset, endDate, startDate = '2019-01-01', method = ''):
    
        dataset = dataset.loc[startDate:endDate, ]
        if method == 'all':
            portfolio = ''
            sharpe = SharpeRatioT(dataset)
        if method == '' or method == 'km' or method == 'hc':
            data = returnAndVolatility(dataset, startDate, endDate)
            if method == 'km' or method == 'hc':
                bestSharpeRatio = 0
                numberCluster = 0

                for n_cluster in range(10, 40):
                    portfolio = selectAssets(data, n_cluster, method)
                    bestSharpeRatioStep = SharpeRatioT(dataset, portfolio)

                    if bestSharpeRatioStep >= bestSharpeRatio:
                        bestSharpeRatio = bestSharpeRatioStep
                        numberCluster = n_cluster

                #print ('Best numberCluster:', numberCluster)
                portfolio = selectAssets(data, numberCluster, method)
                sharpe = SharpeRatioT(dataset, portfolio)
            if method == '':
                portfolio = selectAssets(data)
                sharpe = SharpeRatioT(dataset, portfolio)
        return round(sharpe, 3)

In [44]:
def RebalanceSharpeT(dataset, period, method):
    sharpeRatio = []
    for i in range(len(period) - 1):
        sharpe = bestSharpeRatioIndexT(dataset, method = method, endDate = period[i + 1])
        sharpeRatio.append(sharpe)
    return sharpeRatio

In [45]:
resultsIncomeT = {
    'km': {
        'm1': [1000, 976, 992, 983, 937, 932, 955, 951, 909, 820, 731, 831, 898],
        'm2': [1000, 948, 908, 947, 901, 695, 925],
        'm3': [1000, 964, 1071, 919, 969],
        'm4': [1000, 944, 1151, 1171],
        'm6': [1000, 1008, 965]
    },
    'hc': {
        'm1': [1000, 967, 949, 941, 862, 1022, 1108, 1148, 1079, 1030, 865, 1004, 1060],
        'm2': [1000, 936, 820, 971, 933, 729, 922],
        'm3': [1000, 959, 1074, 935, 1031],
        'm4': [1000, 936, 1017, 1016],
        'm6': [1000, 1004, 1061]
    },
    'ap': {
        'm1': [1000, 895, 869, 860, 695, 737, 785, 779, 746, 697, 580, 655, 714],
        'm2': [1000, 863, 796, 839, 801, 651, 812],
        'm3': [1000, 872, 720, 647, 685],
        'm4': [1000, 782, 984, 975],
        'm6': [1000, 792, 793]
    },
    'all': {
        'm1': [1000, 953, 969, 973, 897, 931, 1012, 1012, 970, 898, 734, 853, 965],
        'm2': [1000, 931, 857, 904, 853, 623, 838],
        'm3': [1000, 950, 1028, 915, 992],
        'm4': [1000, 910, 922, 930],
        'm6': [1000, 945, 923]
    }
}

In [46]:
resultsSharpeT = {
    'km': {
        'm1': [2.413, 2.203, 2.073, 2.107, 1.956, 2.0, 1.994, 1.96, 1.955, 1.851, 1.694, 1.774, 1.862],
        'm2': [2.413, 2.073, 1.956, 1.994, 1.955, 1.694, 1.862],
        'm3': [2.413, 2.107, 1.994, 1.851, 1.862],
        'm4': [2.413, 1.956, 1.955, 1.862],
        'm6': [2.413, 1.994, 1.862]
    },
    'hc': {
        'm1': [2.404, 2.119, 1.991, 1.972, 1.898, 1.953, 1.977, 1.958, 1.955, 1.834, 1.689, 1.763, 1.855],
        'm2': [2.404, 1.991, 1.898, 1.977, 1.955, 1.689, 1.855],
        'm3': [2.404, 1.972, 1.977, 1.834, 1.855],
        'm4': [2.404, 1.898, 1.955, 1.855],
        'm6': [2.404, 1.977, 1.855]
    },
    'ap': {
        'm1': [2.404, 2.117, 1.991, 1.754, 1.956, 1.869, 1.975, 1.958, 1.935, 1.834, 1.685, 1.643, 1.847],
        'm2': [2.404, 1.991, 1.956, 1.975, 1.935, 1.685, 1.847],
        'm3': [2.404, 1.754, 1.975, 1.834, 1.847],
        'm4': [2.404, 1.956, 1.935, 1.847],
        'm6': [2.404, 1.975, 1.847]
    },
    'all': {
        'm1': [2.419, 2.215, 2.141, 2.116, 2.017, 2.041, 2.049, 2.01, 1.996, 1.882, 1.743, 1.837, 1.903],
        'm2': [2.419, 2.141, 2.017, 2.049, 1.996, 1.743, 1.903],
        'm3': [2.419, 2.116, 2.049, 1.882, 1.903],
        'm4': [2.419, 2.017, 1.996, 1.903],
        'm6': [2.419, 2.049, 1.903]
    }
}

In [47]:
def plot_portfolio_values(dataset, initial, period, methods):
    # Prepare the data
    x = period[1:]
    
    # Convert the date strings to datetime objects
    x_dates = [datetime.strptime(date, '%Y-%m-%d') for date in x]

    # Colors for the lines
    colors = ['red', 'blue', 'green', 'yellow']

    # Dictionary to store results for each method
    results = {}

    # Compute results for each method
    for method in methods:
        results[method] = portfolioRebalanceIncomeT(dataset, initial, period, method)

    # Create the plot
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot data for each method
    for method, color in zip(methods, colors):
        y = results[method]
        ax.plot(x_dates, y, marker='o', linestyle='-', color=color, label=f'Method: {method}')
    
    # Formatting the plot
    ax.set_title('Portfolio Values Over Time', fontsize=16)
    ax.set_xlabel('Date', fontsize=14)
    ax.set_ylabel('Value', fontsize=14)
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.legend()

    # Formatting the date on the x-axis
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)

    # Adding a grid
    ax.grid(True)

    # Show the plot
    plt.tight_layout()
    plt.show()

In [48]:
def plot_sharpe_ratio(dataset, period, methods):
    # Prepare the data
    x = period[1:]
    
    # Convert the date strings to datetime objects
    x_dates = [datetime.strptime(date, '%Y-%m-%d') for date in x]

    # Colors for the lines
    colors = ['red', 'blue', 'green', 'yellow']

    # Dictionary to store results for each method
    results = {}

    # Compute results for each method
    for method in methods:
        results[method] = RebalanceSharpeT(dataset, period, method)

    # Create the plot
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot data for each method
    for method, color in zip(methods, colors):
        y = results[method]
        ax.plot(x_dates[:len(y)], y, marker='o', linestyle='-', color=color, label=f'Method: {method}')
    
    # Formatting the plot
    ax.set_title('Sharpe Ratio Over Time', fontsize=16)
    ax.set_xlabel('Date', fontsize=14)
    ax.set_ylabel('Value', fontsize=14)
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.legend()

    # Formatting the date on the x-axis
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)

    # Adding a grid
    ax.grid(True)

    # Show the plot
    plt.tight_layout()
    plt.show()

In [49]:
def ReturnVolatilityT(dataset, portfolio = '', transaction_cost_pct=0.01):
    if portfolio == '':
        df = dataset
    else:
        portfolio = [x for x in portfolio if x is not None]
        df = dataset.iloc[:, portfolio]

    # Calculate the expected returns and the annualized sample covariance matrix of asset returns
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)

    ef = EfficientFrontier(mu, S)  # Set the solver here
    
    ef.add_objective(transaction_cost_objective, S=S, transaction_cost_pct=transaction_cost_pct)
    
    weights = ef.max_sharpe()

    cleaned_weights = ef.clean_weights()
    cleaned_weights = {asset: round(weight, 2) for asset, weight in weights.items() if weight > 0.01}
    #print(cleaned_weights) 
    #print('Number of Assets:',  len(cleaned_weights))
    # to exit Sharp Ratio r is that tuple
    r = []
    r = ef.portfolio_performance()
    Return, Volatility = r[0:2]
    return Return, Volatility

In [50]:
def ReturnVolatilityIndexT(dataset, endDate, startDate = '2019-01-01', method = ''):
    
        dataset = dataset.loc[startDate:endDate, ]
        if method == 'all':
            portfolio = ''
            Return, Volatility = ReturnVolatilityT(dataset)
        if method == '' or method == 'km' or method == 'hc':
            data = returnAndVolatility(dataset, startDate, endDate)
            if method == 'km' or method == 'hc':
                bestSharpeRatio = 0
                numberCluster = 0

                for n_cluster in range(10, 40):
                    portfolio = selectAssets(data, n_cluster, method)
                    bestSharpeRatioStep = SharpeRatioT(dataset, portfolio)

                    if bestSharpeRatioStep >= bestSharpeRatio:
                        bestSharpeRatio = bestSharpeRatioStep
                        numberCluster = n_cluster

                #print ('Best numberCluster:', numberCluster)
                portfolio = selectAssets(data, numberCluster, method)
                Return, Volatility = ReturnVolatilityT(dataset, portfolio)
            if method == '':
                portfolio = selectAssets(data)
                Return, Volatility = ReturnVolatilityT(dataset, portfolio)
        return Return, Volatility

In [51]:
def RebalanceReturnVolatilityT(dataset, period, method):
    d = len(period) - 2
    Return_list = []
    Volatility_list = []
    for i in range(len(period) - 1):
        current_return, current_volatility = ReturnVolatilityIndexT(dataset, method=method, endDate=period[i + 1])
        
        Return_list.append(round(current_return / d, 3))
        Volatility_list.append(round(current_volatility / d, 3))
    return Return_list, Volatility_list

In [52]:
resultsReturnT = {
    'km': {
        'm1': [0.066, 0.06, 0.055, 0.053, 0.047, 0.05, 0.047, 0.046, 0.047, 0.044, 0.04, 0.041, 0.043],
        'm2': [0.131, 0.11, 0.095, 0.095, 0.094, 0.079, 0.085],
        'm3': [0.197, 0.158, 0.142, 0.131, 0.128],
        'm4': [0.263, 0.189, 0.188, 0.171],
        'm6': [0.394, 0.284, 0.256]
    },
    'hc': {
        'm1': [0.066, 0.075, 0.071, 0.059, 0.047, 0.051, 0.051, 0.046, 0.047, 0.045, 0.04, 0.041, 0.043],
        'm2': [0.132, 0.141, 0.094, 0.102, 0.094, 0.08, 0.086],
        'm3': [0.198, 0.176, 0.153, 0.136, 0.129],
        'm4': [0.264, 0.188, 0.188, 0.172],
        'm6': [0.396, 0.307, 0.258]
    },
    'ap': {
        'm1': [0.066, 0.075, 0.071, 0.118, 0.047, 0.064, 0.052, 0.046, 0.049, 0.045, 0.041, 0.041, 0.043],
        'm2': [0.132, 0.141, 0.095, 0.105, 0.098, 0.081, 0.086],
        'm3': [0.198, 0.353, 0.157, 0.136, 0.129],
        'm4': [0.264, 0.19, 0.195, 0.173],
        'm6': [0.396, 0.315, 0.259]
    },
    'all': {
        'm1': [0.064, 0.058, 0.054, 0.051, 0.043, 0.047, 0.048, 0.047, 0.048, 0.044, 0.039, 0.041, 0.043],
        'm2': [0.129, 0.109, 0.086, 0.096, 0.095, 0.077, 0.087],
        'm3': [0.193, 0.154, 0.144, 0.132, 0.13],
        'm4': [0.257, 0.173, 0.19, 0.174],
        'm6': [0.386, 0.287, 0.26]
    }
}

In [53]:
resultsVolatilityT = {
    'km': {
        'm1': [0.027, 0.026, 0.026, 0.024, 0.023, 0.024, 0.023, 0.023, 0.023, 0.023, 0.022, 0.022, 0.022],
        'm2': [0.053, 0.052, 0.047, 0.046, 0.046, 0.045, 0.044],
        'm3': [0.08, 0.072, 0.069, 0.068, 0.066],
        'm4': [0.106, 0.093, 0.093, 0.088],
        'm6': [0.159, 0.138, 0.132]
    },
    'hc': {
        'm1': [0.027, 0.035, 0.035, 0.029, 0.024, 0.025, 0.025, 0.023, 0.023, 0.024, 0.023, 0.022, 0.022],
        'm2': [0.054, 0.069, 0.048, 0.05, 0.046, 0.045, 0.044],
        'm3': [0.08, 0.087, 0.075, 0.072, 0.067],
        'm4': [0.107, 0.095, 0.093, 0.089],
        'm6': [0.161, 0.15, 0.133]
    },
    'ap': {
        'm1': [0.027, 0.035, 0.035, 0.066, 0.023, 0.034, 0.026, 0.023, 0.024, 0.024, 0.023, 0.024, 0.022],
        'm2': [0.054, 0.069, 0.047, 0.051, 0.049, 0.046, 0.045],
        'm3': [0.08, 0.198, 0.077, 0.072, 0.067],
        'm4': [0.107, 0.094, 0.098, 0.09],
        'm6': [0.161, 0.154, 0.135]
    },
    'all': {
        'm1': [0.026, 0.025, 0.025, 0.023, 0.021, 0.022, 0.023, 0.023, 0.023, 0.022, 0.021, 0.021, 0.022],
        'm2': [0.052, 0.049, 0.041, 0.045, 0.046, 0.042, 0.044],
        'm3': [0.078, 0.07, 0.068, 0.067, 0.066],
        'm4': [0.104, 0.082, 0.092, 0.088],
        'm6': [0.156, 0.135, 0.132]
    }
}